In [1]:
import adjust_path
import sys
from icc import data_loader
sys.path

Appending /home/milesg/Projects/StatoilCCoreKaggle/notebooks/.. to path


['/home/milesg/Projects/StatoilCCoreKaggle/notebooks/..',
 '',
 '/home/milesg/anaconda/lib/python27.zip',
 '/home/milesg/anaconda/lib/python2.7',
 '/home/milesg/anaconda/lib/python2.7/plat-linux2',
 '/home/milesg/anaconda/lib/python2.7/lib-tk',
 '/home/milesg/anaconda/lib/python2.7/lib-old',
 '/home/milesg/anaconda/lib/python2.7/lib-dynload',
 '/home/milesg/anaconda/lib/python2.7/site-packages',
 '/home/milesg/anaconda/lib/python2.7/site-packages/setuptools-27.2.0-py2.7.egg',
 '/home/milesg/anaconda/lib/python2.7/site-packages/IPython/extensions',
 '/home/milesg/.ipython']